## Import

In [161]:
from tqdm import tqdm

import streamlit as st
import pandas as pd
import numpy as np
import OpenDartReader
import warnings
import dart_fss
import time
import re, os

warnings.filterwarnings('ignore')

* functions

In [2]:
def get_data(dart, code, year = 2022, quarter = "사업보고서"):
    select_cols = ['corp_cls', 'corp_code', 'corp_name', 'inv_prm', 'frst_acqs_de', 'invstmnt_purps', 'frst_acqs_amount', 'trmend_blce_qy', 'trmend_blce_qota_rt', 'trmend_blce_acntbk_amount']
    change_cols = ['법인구분', '고유번호', '회사명', '법인명', '최초취득일자', '출자목적', '최초취득금액', '기말잔액수량', '기말잔액지분율', '기말잔액장부가액']
    change_cls = {"Y":"유가", "K":"코스닥", "N":"코넥스", "E":"기타"}
    
    change_dict = {"1분기보고서": 11013, "반기보고서": 11012, "3분기보고서": 11014, "사업보고서":11011}
    r_code = change_dict[quarter]
    
    invst_df = dart.report(code, '타법인출자', year, r_code)
    
    if invst_df.shape[0] == 0:
        return invst_df
    else:
        invst_df = invst_df.loc[:, select_cols]
        invst_df.corp_cls = invst_df.corp_cls.map(change_cls)
        invst_df.columns = change_cols
    
        return invst_df

* API Setting

In [162]:
api_key = '1b39652cef07f626c9d37375edf582ee51b1407f'
dart = OpenDartReader(api_key)
dart_fss.set_api_key(api_key=api_key)

'1b39652cef07f626c9d37375edf582ee51b1407f'

* get formatting data

In [ ]:
# 발행실적 보고서에 올라와있는 기업들만 검색하자

In [169]:
temp_df = dart.regstate(corp, '지분증권')

In [170]:
origin_feats = ['rcept_no', 'corp_cls', 'corp_code', 'corp_name', 'sbd', 'pymd', 'sband', 'asand', 'asstd', 'exstk', 'exprc', 'expd', 
                'rpt_rcpn', 'title', 'stksen', 'stkcnt', 'fv', 'slprc', 'slta', 'slmthn', 'actsen',
                'actnmn', 'udtcnt', 'udtamt', 'udtprc', 'udtmth', 'se', 'amt', 'hdr',
                'rl_cmp', 'bfsl_hdstk', 'slstk', 'atsl_hdstk', 'grtrs', 'exavivr', 'grtcnt']

change_feats = ['접수번호', '법인구분', '고유번호', '회사명', '청약기일', '납입기일', '청약공고일', '배정공고일', '배정기준일', '행사대상증권','행사가격', '행사기간' ,
                '주요사항보고서(접수번호)', '그룹명칭', '증권의종류', '증권수량', '액면가액', '모집(매출)가액', '모집(매출)총액', '모집(매출)방법', '인수인구분', 
                '인수인명', '인수수량', '인수금액', '인수대가', '인수방법', '구분', '금액', '보유자',
                '회사와의관계', '매출전보유증권수', '매출증권수', '매출후보유증권수', '부여사유', '행사가능투자자', '부여수량']

change_dict = {x:y for x, y in zip(origin_feats, change_feats)}
temp_df.columns = [change_dict[x] for x in temp_df.columns]

In [171]:
print(change_feats)

['접수번호', '법인구분', '고유번호', '회사명', '청약기일', '납입기일', '청약공고일', '배정공고일', '배정기준일', '행사대상증권', '행사가격', '행사기간', '주요사항보고서(접수번호)', '그룹명칭', '증권의종류', '증권수량', '액면가액', '모집(매출)가액', '모집(매출)총액', '모집(매출)방법', '인수인구분', '인수인명', '인수수량', '인수금액', '인수대가', '인수방법', '구분', '금액', '보유자', '회사와의관계', '매출전보유증권수', '매출증권수', '매출후보유증권수', '부여사유', '행사가능투자자', '부여수량']


In [172]:
temp_df2 = dart.regstate('케이옥션', '지분증권')
temp_df2.columns = [change_dict[x] for x in temp_df2.columns]

In [173]:
temp_df.shape

(9, 36)

In [312]:
bgn_de, end_de = '20230330', '20230425'

In [319]:
def get_info(bgn_de, end_de) :
    rcept_info = []

    url = 'https://opendart.fss.or.kr/api/list.xml'
    params = {'crtfc_key': api_key
              , 'bgn_de' : bgn_de
              , 'end_de' : end_de
              , 'pblntf_detail_ty': 'C001' # 증권신고(지분증권)
              , 'last_reprt_at':'Y'} # 최종보고서 검색여부
    response = requests.get(url, params=params, verify=False)
    soup = BeautifulSoup(response.content, features='xml')
    total_page = soup.find('total_page').get_text()

    for i in range(1, int(total_page) + 1):
        params = {'crtfc_key': api_key
                  , 'bgn_de': bgn_de
                  , 'end_de': end_de
                  , 'page_no': str(i)
                  , 'pblntf_detail_ty': 'C001' # 증권신고(지분증권)
                  , 'last_reprt_at':'Y'} # 최종보고서 검색여부
        response = requests.get(url, params=params, verify=False)
        soup = BeautifulSoup(response.content, features='xml')
        rcept_info.extend([x for x in soup.find_all('list') if "증권발행실적보고서" in x.get_text()])
    print('보고서 수 :', len(rcept_info))
    return rcept_info

In [320]:
rcept_info = get_info(bgn_de, end_de)

보고서 수 : 7


In [326]:
[x.find('rcept_no').get_text() for x in rcept_info]

['20230421000213',
 '20230420000424',
 '20230420000289',
 '20230413002343',
 '20230331002508',
 '20230330000951',
 '20230330000849']

In [328]:
[x.find('corp_name').get_text() for x in rcept_info]

['토마토시스템', '젠큐릭스', '마이크로투나노', '비츠로시스', '미래에셋비전스팩3호', '하나27호스팩', 'IBKS제22호스팩']

In [330]:
url = 'https://opendart.fss.or.kr/api/document.xml'
params = {'crtfc_key': api_key, 'rcept_no': '20230421000213'}
response = requests.get(url, params=params)

In [342]:
zf = zipfile.ZipFile(BytesIO(response.content))
fp = zf.read('{}.xml'.format('20230421000213'))

In [344]:
xml_str = fp.decode('utf-8')
xml_str = xml_str.replace('<=', '')
xml = xml_str.encode('utf-8')

In [345]:
soup = BeautifulSoup(xml, features='html.parser')

In [360]:
table = soup.find('table-group', attrs={'aclass': 'PBL_OTLN'})

In [374]:
table.find_all('table')

[<table aclass="EXTRACTION" afixtable="Y" border="0" width="600">
 <colgroup>
 <col width="291"/>
 <col width="291"/>
 </colgroup>
 <tbody>
 <tr acopy="N" adelete="N">
 <td height="30" valign="BOTTOM" width="300"></td>
 <tu align="RIGHT" aunit="WON" aunitvalue="1" height="30" valign="BOTTOM" width="300">(단위 : 원)</tu>
 </tr>
 </tbody>
 </table>,
 <table aclass="EXTRACTION" afixtable="Y" border="1" width="600">
 <colgroup>
 <col width="191"/>
 <col width="191"/>
 <col width="191"/>
 </colgroup>
 <thead align="LEFT" valign="TOP">
 <tr acopy="Y" adelete="Y">
 <th acopycol="N" adeletecol="N" align="CENTER" amovecol="N" aupdatecont="N" height="30" valign="MIDDLE" width="200">구분</th>
 <th acopycol="N" adeletecol="N" align="CENTER" amovecol="N" aupdatecont="N" height="30" valign="MIDDLE" width="200">발행총액</th>
 <th acopycol="N" adeletecol="N" align="CENTER" amovecol="N" aupdatecont="N" height="30" valign="MIDDLE" width="200">비고</th>
 </tr>
 </thead>
 <tbody align="LEFT" valign="TOP">
 <tr acopy

In [392]:
temp_txt = str(table.find_all('table')[1]); temp_txt

'<table aclass="EXTRACTION" afixtable="Y" border="1" width="600">\n<colgroup>\n<col width="191"/>\n<col width="191"/>\n<col width="191"/>\n</colgroup>\n<thead align="LEFT" valign="TOP">\n<tr acopy="Y" adelete="Y">\n<th acopycol="N" adeletecol="N" align="CENTER" amovecol="N" aupdatecont="N" height="30" valign="MIDDLE" width="200">구분</th>\n<th acopycol="N" adeletecol="N" align="CENTER" amovecol="N" aupdatecont="N" height="30" valign="MIDDLE" width="200">발행총액</th>\n<th acopycol="N" adeletecol="N" align="CENTER" amovecol="N" aupdatecont="N" height="30" valign="MIDDLE" width="200">비고</th>\n</tr>\n</thead>\n<tbody align="LEFT" valign="TOP">\n<tr acopy="N" adelete="N">\n<tu align="CENTER" aunit="DST_SMR" aunitvalue="1" height="30" valign="MIDDLE" width="200">기업공개</tu>\n<te acode="INC_SL_PO_SUM" align="CENTER" height="30" valign="MIDDLE" width="200">15,652,000,000</te>\n<te acode="NOTE" align="CENTER" height="30" valign="MIDDLE" width="200">-</te>\n</tr>\n</tbody>\n</table>'

In [393]:
pd.read_html(temp_txt.replace('te', 'td').replace('tu', 'td'))

[     구분         발행총액 비고
 0  기업공개  15652000000  -]

In [394]:
temp_txt = str(table.find_all('table')[0]); temp_txt

'<table aclass="EXTRACTION" afixtable="Y" border="0" width="600">\n<colgroup>\n<col width="291"/>\n<col width="291"/>\n</colgroup>\n<tbody>\n<tr acopy="N" adelete="N">\n<td height="30" valign="BOTTOM" width="300"></td>\n<tu align="RIGHT" aunit="WON" aunitvalue="1" height="30" valign="BOTTOM" width="300">(단위 : 원)</tu>\n</tr>\n</tbody>\n</table>'

In [402]:
pd.read_html(temp_txt)

[]

In [401]:
pd.read_html(temp_txt.replace('te', 'td').replace('tu', 'td'))

[    0         1
 0 NaN  (단위 : 원)]

In [361]:
table

<table-group aclass="PBL_OTLN" adeletetable="N">
<table aclass="EXTRACTION" afixtable="Y" border="0" width="600">
<colgroup>
<col width="291"/>
<col width="291"/>
</colgroup>
<tbody>
<tr acopy="N" adelete="N">
<td height="30" valign="BOTTOM" width="300"></td>
<tu align="RIGHT" aunit="WON" aunitvalue="1" height="30" valign="BOTTOM" width="300">(단위 : 원)</tu>
</tr>
</tbody>
</table>
<table aclass="EXTRACTION" afixtable="Y" border="1" width="600">
<colgroup>
<col width="191"/>
<col width="191"/>
<col width="191"/>
</colgroup>
<thead align="LEFT" valign="TOP">
<tr acopy="Y" adelete="Y">
<th acopycol="N" adeletecol="N" align="CENTER" amovecol="N" aupdatecont="N" height="30" valign="MIDDLE" width="200">구분</th>
<th acopycol="N" adeletecol="N" align="CENTER" amovecol="N" aupdatecont="N" height="30" valign="MIDDLE" width="200">발행총액</th>
<th acopycol="N" adeletecol="N" align="CENTER" amovecol="N" aupdatecont="N" height="30" valign="MIDDLE" width="200">비고</th>
</tr>
</thead>
<tbody align="LEFT" va

In [356]:
soup.find_all('table-group')

[<table-group aclass="CRP_OTLN" adeletetable="N">
 <table aclass="EXTRACTION" afixtable="Y" border="1" width="600">
 <colgroup>
 <col width="191"/>
 <col width="191"/>
 <col width="191"/>
 </colgroup>
 <thead align="LEFT" valign="TOP">
 <tr acopy="Y" adelete="Y">
 <th acopycol="N" adeletecol="N" align="CENTER" amovecol="N" aupdatecont="N" height="30" valign="MIDDLE" width="200">상장구분</th>
 <th acopycol="N" adeletecol="N" align="CENTER" amovecol="N" aupdatecont="N" height="30" valign="MIDDLE" width="200">기업규모</th>
 <th acopycol="N" adeletecol="N" align="CENTER" amovecol="N" aupdatecont="N" height="30" valign="MIDDLE" width="200">업종구분</th>
 </tr>
 </thead>
 <tbody align="LEFT" valign="TOP">
 <tr acopy="N" adelete="N">
 <tu align="CENTER" aunit="CRP_RGS_CLS" aunitvalue="2" height="53" valign="MIDDLE" width="200">코스닥</tu>
 <tu align="CENTER" aunit="CRP_SCL" aunitvalue="2" height="53" valign="MIDDLE" width="200">중소기업</tu>
 <tu align="CENTER" aunit="CRP_CODE" aunitvalue="62" height="53" valig

In [339]:
soup = BeautifulSoup(response.content, features='xml')

In [340]:
soup

<?xml version="1.0" encoding="utf-8"?>

In [276]:
from io import BytesIO
import zipfile

zf = zipfile.ZipFile(BytesIO(response.content))
z_list = zf.namelist()
file = zf.read(z_list[0]) 

soup = BeautifulSoup(file, 'html.parser', from_encoding='utf-8')
table = soup.find('table-group', attrs={'aclass':'PL_KND_WRT'})

BadZipFile: File is not a zip file

In [252]:
soup

<?xml version="1.0" encoding="utf-8"?>
<document xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:nonamespaceschemalocation="dart3.xsd">
<document-name acode="10800">증권발행실적보고서</document-name>
<formula-version adate="20210701" subver="1">2.5</formula-version>
<company-name aregcik="01591262">디비금융제10호 기업인수목적 주식회사</company-name>
<body>
<cover>
<cover-title aassocnote="COVER" atoc="Y">증권발행실적보고서</cover-title>
<table aclass="NORMAL" afixtable="N" border="0" width="600">
<colgroup>
<col width="152"/>
<col width="430"/>
</colgroup>
<tbody>
<tr acopy="Y" adelete="Y">
<td class="NORMAL" colspan="2" height="53" width="600">&amp;cr</td>
</tr>
<tr acopy="Y" adelete="Y">
<td class="NORMAL" height="30" width="161">금융감독원장 귀하</td>
<td align="RIGHT" class="NORMAL" height="30" width="439">  2022 년    01 월    13 일</td>
</tr>
<tr acopy="Y" adelete="Y">
<td class="NORMAL" height="53" width="161"></td>
<td class="NORMAL" height="53" width="439">&amp;cr</td>
</tr>
<tr acopy="Y" adelete="Y">
<td class

In [ ]:
for c in soup.find_all('list'):
    if c.report_nm.get_text() in rcept_names:
        if c.report_nm.get_text()=='[기재정정]증권신고서(채무증권)':
            rcept_info.append(c.rcept_no.get_text()+'_'+c.corp_code.get_text()+'_'+c.corp_name.get_text()+'_정정')
        else : 
            rcept_info.append(c.rcept_no.get_text()+'_'+c.corp_code.get_text()+'_'+c.corp_name.get_text()+'_최초')

In [223]:
response = requests.get(url, params=params, verify=False)
soup = BeautifulSoup(response.content, features='xml')

In [ ]:
rcept_names = ['증권신고서(지분증권)','[첨부추가]증권신고서(지분증권)','[첨부정정]증권신고서(지분증권)','[기재정정]증권신고서(지분증권)']

In [214]:
for i in range(1, int(total_page) + 1):
    params = {'crtfc_key': api_key
          , 'bgn_de': bgn_de
          , 'end_de': end_de
          , 'page_no': str(i)
          , 'pblntf_detail_ty': 'C001' # 증권신고(지분증권)
          , 'last_reprt_at':'Y'} # 최종보고서 검색여부
    
    response = requests.get(url, params=params, verify=False)
    soup = BeautifulSoup(response.content, features='xml')
    rcept_names = ['증권신고서(채무증권)','[첨부추가]증권신고서(채무증권)','[첨부정정]증권신고서(채무증권)','[기재정정]증권신고서(채무증권)']
    for c in soup.find_all('list'):
        if c.report_nm.get_text() in rcept_names:
            if c.report_nm.get_text()=='[기재정정]증권신고서(채무증권)':
                rcept_info.append(c.rcept_no.get_text()+'_'+c.corp_code.get_text()+'_'+c.corp_name.get_text()+'_정정')
            else : 
                rcept_info.append(c.rcept_no.get_text()+'_'+c.corp_code.get_text()+'_'+c.corp_name.get_text()+'_최초')
print('보고서 수 : ', len(rcept_info))  
st.write('보고서 수 : ', len(rcept_info)) 

In [210]:
from bs4 import BeautifulSoup

In [211]:
get_info('20220101', '20220131')

보고서 수 :  26


['20220128000144_01152586_유일로보틱스_최초',
 '20220127000204_01305249_스톤브릿지벤처스_정정',
 '20220127000165_01150515_대명에너지_최초',
 '20220126000500_01354740_브이씨_정정',
 '20220126000228_01462605_지투파워_최초',
 '20220125000396_01423837_모아데이타_최초',
 '20220125000302_01357446_퓨런티어_정정',
 '20220124000165_01327746_보로노이_최초',
 '20220120000260_01596656_메쎄이상_최초',
 '20220120000259_01301623_비씨엔씨_최초',
 '20220119000131_01494118_노을_최초',
 '20220114000441_00922304_아셈스_정정',
 '20220114000247_01160974_바이오에프디엔씨_정정',
 '20220113000186_00922304_아셈스_정정',
 '20220113000095_01593242_하나금융21호스팩_최초',
 '20220111000122_00417167_풍원정밀_최초',
 '20220110000490_01290248_스코넥_정정',
 '20220107000521_01305249_스톤브릿지벤처스_최초',
 '20220107000512_01591606_IBKS제17호스팩_정정',
 '20220107000231_01357446_퓨런티어_최초',
 '20220107000213_01354740_브이씨_최초',
 '20220107000137_01591262_디비금융스팩10호_최초',
 '20220106000097_00362122_나래나노텍_정정',
 '20220105000395_01140837_이지트로닉스_정정',
 '20220105000203_00362122_나래나노텍_정정',
 '20220104000136_00668600_케이옥션_정정']

In [177]:
start = None
end = None

corp_code = '00922304'
key_word = '지분증권'

In [179]:
start = pd.to_datetime(start) if start else pd.to_datetime('1900-01-01')
end = pd.to_datetime(end) if end else datetime.today()

key_word_map = {
    '주식의포괄적교환이전': 'extrRs', # 주식의포괄적교환이전
    '합병': 'mgRs', # 합병
    '증권예탁증권': 'stkdpRs', # 증권예탁증권
    '채무증권': 'bdRs', #  채무증권
    '지분증권': 'estkRs', # 지분증권 
    '분할': 'dvRs', #  분할
} 

url = f"https://opendart.fss.or.kr/api/{key_word_map[key_word]}.json"
params = {
    'crtfc_key': api_key,    # 인증키
    'corp_code': corp_code,   # 회사 고유번호
    'bgn_de': start.strftime('%Y%m%d'),  # 시작일(최초접수일)
    'end_de': end.strftime('%Y%m%d'), # 종료일(최초접수일)
    'last_reprt_at':'Y'
}
r = requests.get(url, params=params)
jo = r.json()

In [181]:
if jo['status'] != '000':
    print(jo)
    output =  pd.DataFrame()
elif 'list' in jo:
    output =  pd.DataFrame(jo['list'])
elif 'group' in jo:
    group = pd.DataFrame(jo['group'])
    df_list = []
    for ix, row in group.iterrows():
        inner_df = pd.DataFrame(row['list'])
        inner_df['title'] = row['title']
        df_list.append(inner_df)
    output =  pd.concat(df_list)
else:
    print('invalid respose')
    print(jo)
    output =  pd.DataFrame()

In [183]:
output.iloc[:, :18]

,rcept_no,corp_cls,corp_code,corp_name,sbd,pymd,sband,asand,asstd,exstk,exprc,expd,rpt_rcpn,title,stksen,stkcnt,fv,slprc
0,20220114000441,K,00922304,아셈스,2022년 01월 24일 ~ 2022년 01월 25일,2022년 01월 27일,2022년 01월 24일,2022년 01월 27일,-,-,-,-,-,일반사항,NaN,NaN,NaN,NaN
0,20220114000441,K,00922304,아셈스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,증권의종류,기명식보통주,"2,000,000",500,"7,000"
0,20220114000441,K,00922304,아셈스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,인수인정보,기명식보통주,NaN,NaN,NaN
0,20220114000441,K,00922304,아셈스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,자금의사용목적,NaN,NaN,NaN,NaN
1,20220114000441,K,00922304,아셈스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,자금의사용목적,NaN,NaN,NaN,NaN
2,20220114000441,K,00922304,아셈스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,자금의사용목적,NaN,NaN,NaN,NaN
3,20220114000441,K,00922304,아셈스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,자금의사용목적,NaN,NaN,NaN,NaN
0,20220114000441,K,00922304,아셈스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,매출인에관한사항,NaN,NaN,NaN,NaN
0,20220114000441,K,00922304,아셈스,NaN,NaN,NaN,NaN,NaN,NaN,-,-,NaN,일반청약자환매청구권,NaN,NaN,NaN,NaN


In [184]:
output.iloc[:, 18:]

,slta,slmthn,actsen,actnmn,udtcnt,udtamt,udtprc,udtmth,se,amt,hdr,rl_cmp,bfsl_hdstk,slstk,atsl_hdstk,grtrs,exavivr,grtcnt
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"14,000,000,000",일반공모,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,대표,한국투자증권,"2,000,000","14,000,000,000","576,800,000",총액인수,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,시설자금,"6,041,000,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해외투자자금,"2,000,000,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,운영자금,"5,735,436,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,발행제비용,"643,564,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,-,-,-,-,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,-,-


In [174]:
temp_df.iloc[:, :18]

,접수번호,법인구분,고유번호,회사명,청약기일,납입기일,청약공고일,배정공고일,배정기준일,행사대상증권,행사가격,행사기간,주요사항보고서(접수번호),그룹명칭,증권의종류,증권수량,액면가액,모집(매출)가액
0,20220114000441,K,00922304,아셈스,2022년 01월 24일 ~ 2022년 01월 25일,2022년 01월 27일,2022년 01월 24일,2022년 01월 27일,-,-,-,-,-,일반사항,NaN,NaN,NaN,NaN
0,20220114000441,K,00922304,아셈스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,증권의종류,기명식보통주,"2,000,000",500,"7,000"
0,20220114000441,K,00922304,아셈스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,인수인정보,기명식보통주,NaN,NaN,NaN
0,20220114000441,K,00922304,아셈스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,자금의사용목적,NaN,NaN,NaN,NaN
1,20220114000441,K,00922304,아셈스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,자금의사용목적,NaN,NaN,NaN,NaN
2,20220114000441,K,00922304,아셈스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,자금의사용목적,NaN,NaN,NaN,NaN
3,20220114000441,K,00922304,아셈스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,자금의사용목적,NaN,NaN,NaN,NaN
0,20220114000441,K,00922304,아셈스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,매출인에관한사항,NaN,NaN,NaN,NaN
0,20220114000441,K,00922304,아셈스,NaN,NaN,NaN,NaN,NaN,NaN,-,-,NaN,일반청약자환매청구권,NaN,NaN,NaN,NaN


In [175]:
temp_df.iloc[:, 18:]

,모집(매출)총액,모집(매출)방법,인수인구분,인수인명,인수수량,인수금액,인수대가,인수방법,구분,금액,보유자,회사와의관계,매출전보유증권수,매출증권수,매출후보유증권수,부여사유,행사가능투자자,부여수량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"14,000,000,000",일반공모,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,대표,한국투자증권,"2,000,000","14,000,000,000","576,800,000",총액인수,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,시설자금,"6,041,000,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해외투자자금,"2,000,000,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,운영자금,"5,735,436,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,발행제비용,"643,564,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,-,-,-,-,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,-,-


In [ ]:
rcept_no_list = []
url = 'https://opendart.fss.or.kr/api/list.xml'
params = {'crtfc_key': api_key
    , 'bgn_de': ''
    , 'end_de': end_de
    , 'pblntf_detail_ty': 'B001'
    , 'last_reprt_at': 'Y'}

In [ ]:

response = requests.get(url, params=params, headers=headers, verify=False)
soup = BeautifulSoup(response.content, features='xml')

try:
    total_page = soup.find('total_page').get_text()
    for i in range(1, int(total_page) + 1):
        params = {'crtfc_key': API_KEY
            , 'bgn_de': bgn_de
            , 'end_de': end_de
            , 'pblntf_detail_ty': 'B001'
            , 'page_no': str(i)
            , 'last_reprt_at': 'Y'}
        response = requests.get(url, params=params, headers=headers, verify=False)
        soup = BeautifulSoup(response.content, features='xml')
        for c in soup.find_all('list'):
            if report_nm in c.report_nm.get_text():
                rcept_no_list.append(c.rcept_no.get_text())
except:
    rcept_no_list = []

print('보고서명:', report_nm, ', 보고서수: ', len(rcept_no_list))